# Lua Tutorial

# Contents

1.) Motivation behind using Lua <br>
2.) Variables and data structures <br>
3.) Basic control-flow structures

## Motivation behind using Lua

1.) LuaJit is a faster interpreter. A comparitive study on Fibonaaci 40th number in Python and Lua showed Lua was 20 times faster over Python. 
Execution time in Lua : 2 second
Execution tim in Python : 37.28 seconds

In [9]:
function fib(n)
    if n<2 then return 1 end
    return fib(n-2) + fib(n-1)
end
function fibOuter(n)
    start_time = os.time()
    print(fib(n))
    end_time = os.time()
    print('start time: '   .. start_time .. 's')
    print('end time: '     .. end_time .. 's')
    elapsed_time = os.difftime(end_time,start_time)
    print('time elapsed: ' .. elapsed_time .. 's')
end
fibOuter(40)

165580141	
start time: 1462759362s	
end time: 1462759364s	
time elapsed: 2s	


2.) Lua has a smaller memory footprint. On the above example memory profiling showed that Lua took 2 times less memory Python
Memory footprint in Lua : 927 bytes
Memory footprint in Python : 2136 bytes

Here are some benchmark performance numbers for few examples http://benchmarksgame.alioth.debian.org/u64q/compare.php?lang=lua&lang2=python3

3.) Closures are a very powerful tool in Lua, other scripting languages like Python support anonymous and lambda functions but not closures in their entirety. In Python, the enclosing state variable are read-only.

In [4]:
function generator()
    local i = 0
    return function()
        if i>=5 then return nil 
        else 
            i=i+1
            return i
        end
    end
end

for val in generator() do
    print(val)
end

1	
2	
3	
4	
5	


4.) Light-weight, can be used for embedded systems. For example, the size of python22.dll is 824kb, however, basic vanilla Lua engine, including parser/compiler/interpreter weighs under 100kb.

5.) The FFI interface is very simple to implement any C/C++ extensions

## Variables and Data Structures

In [9]:
local value = 26 --By default number types are double types
str = 'immutable' --Strings are immutable so avoid concatenation
-- Instead use string format
print(string.format('%s %d',str,value))

immutable 26	


In [10]:
-- Lua has garbage collector anything set to nil will be collected
str = nil
collectgarbage()

nil	


#### Lua's basic datastructure are associative arrays, tables. We can store heterogenous type objects in same table.

In [20]:
--create a table
local t = {}
table.insert(t,'1')
t['2']='two'
t.three = '3'
print("printing table object..")
print(t)
print("scanning over table..")
for id, value in pairs(t) do
    print(string.format("%s %s", id, value))
end
print("access table indices..")
print(t[1])
print(t[2]) -- nil because key is string
print(t['2'])
print(t['three'])

printing table object..	
{
  1 : 1
  2 : two
  three : 3
}
scanning over table..	
1 1	
2 two	
three 3	
access table indices..	
1	
nil	
two	
3	


## Basic control-flow structures

In [11]:
-- while construct
local count=0
while count<10 do
    count=count+1
    print(count)
end

1	
2	
3	
4	
5	
6	
7	
8	
9	
10	


In [2]:
-- if-else clause
local value = 100
if value <= 5 then print("Value is between 0-5") 
elseif value <= 10 then print("Value is between 5-10")
else print("Value is greater than 10")
end

Value is greater than 10	


In [4]:
-- for construct
local sum = 0
for i=1, 10 do --lower and upper bounds included
  sum = sum+i
end
print(string.format("sum: %s",sum))

sum: 55	


In [6]:
-- higher-order functions or closures
function hypotenuse(x)
    return function(y) return x*x + y*y end
end

h=hypotenuse(3)
print(h(4))

25	


In [7]:
-- function iterators or generators
list = {10,'eleven',20,'twenty-one',30,'thirty-one'}
function iterate (t)
  local i = 0
  local n = table.getn(t)
  return function ()
         i = i + 1
         if i <= n then return t[i] end
         end
end

for val in iterate(list) do
    print(val)
end

10	
eleven	
20	
twenty-one	
30	
thirty-one	


In [ ]:
-- 